In [1]:
import torch
import torchvision.models as models
import torch.nn as nn
from torchvision import transforms
from PIL import Image

# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the image transformations for grayscale images
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to RGB
    transforms.Resize((224, 224)),                # Resize to 224x224, standard input size for ResNet
    transforms.ToTensor(),                        # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Define function to load and preprocess image
def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image)
    return image.unsqueeze(0)  # Add batch dimension

# Define and load the model
model = models.resnet50(pretrained=False)
num_classes = 14
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Load and adjust the state dictionary
state_dict = torch.load('resnet50_scratch.pth')

# If the state_dict keys have a 'resnet.' prefix, remove it
new_state_dict = {}
for k, v in state_dict.items():
    if k.startswith('resnet.'):
        new_key = k[len('resnet.'):]
        new_state_dict[new_key] = v
    else:
        new_state_dict[k] = v

model.load_state_dict(new_state_dict)
model.to(device)
model.eval()

# Perform inference on an example image
image_path = './test2.png'
image = load_image(image_path).to(device)

with torch.no_grad():
    output = model(image)
    predictions = torch.sigmoid(output)
    predicted_labels = (predictions > 0.5).float()


c:\Users\Matt\anaconda3\envs\final_project\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Matt\anaconda3\envs\final_project\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\Matt\AppData\Local\Temp\ipykernel_43404\186120305.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default valu

In [2]:
# Pathologies
pathologies = [
    "Atelectasis",
    "Cardiomegaly",
    "Consolidation",
    "Edema",
    "Effusion",
    "Emphysema",
    "Fibrosis",
    "Hernia",
    "Infiltration",
    "Pleural_Thickening",
    "Mass",
    "Nodule",
    "Pneumonia",
    "Pneumothorax"
]

# Convert predictions to readable format
predicted_labels = predicted_labels.squeeze().cpu().numpy()
for i, label in enumerate(predicted_labels):
    print(f'{pathologies[i]}: {"Present" if label else "Absent"}')

Atelectasis: Absent
Cardiomegaly: Absent
Consolidation: Absent
Edema: Absent
Effusion: Absent
Emphysema: Absent
Fibrosis: Absent
Hernia: Absent
Infiltration: Absent
Pleural_Thickening: Absent
Mass: Absent
Nodule: Absent
Pneumonia: Absent
Pneumothorax: Absent


In [3]:
import gradio as gr
import torch
from torchvision import models, transforms
from PIL import Image
import torch.nn as nn

# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the image transformations for grayscale images
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to RGB
    transforms.Resize((224, 224)),                # Resize to 224x224, standard input size for ResNet
    transforms.ToTensor(),                        # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Define pathologies
pathologies = [
    "Atelectasis",
    "Cardiomegaly",
    "Consolidation",
    "Edema",
    "Effusion",
    "Emphysema",
    "Fibrosis",
    "Hernia",
    "Infiltration",
    "Pleural_Thickening",
    "Mass",
    "Nodule",
    "Pneumonia",
    "Pneumothorax"
]

# # Load the model
# model = models.resnet50(pretrained=False)
# num_classes = 14
# model.fc = nn.Linear(model.fc.in_features, num_classes)
# model.load_state_dict(torch.load('resnet50_scratch.pth'))
# model.to(device)
# model.eval()

# Load and adjust the state dictionary
state_dict = torch.load('resnet50_scratch.pth')

# If the state_dict keys have a 'resnet.' prefix, remove it
new_state_dict = {}
for k, v in state_dict.items():
    if k.startswith('resnet.'):
        new_key = k[len('resnet.'):]
        new_state_dict[new_key] = v
    else:
        new_state_dict[k] = v

model.load_state_dict(new_state_dict)
model.to(device)
model.eval()

# Define the prediction function
def predict(image):
    image = Image.fromarray(image).convert('RGB')  # Ensure the image is in RGB format
    image = transform(image).unsqueeze(0).to(device)  # Apply transformations and move to GPU
    with torch.no_grad():
        output = model(image)
        predictions = torch.sigmoid(output)
        predicted_labels = (predictions > 0.5).float()
    result = {pathologies[i]: "Present" if predicted_labels[0, i] else "Absent" for i in range(num_classes)}
    return result

# Create the Gradio interface
iface = gr.Interface(fn=predict, inputs=gr.Image(type="numpy"), outputs=gr.JSON())
iface.launch()


c:\Users\Matt\anaconda3\envs\final_project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Matt\AppData\Local\Temp\ipykernel_43404\1280477760.py:45: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
